In [5]:
import pandas as pd
import numpy as np

In [44]:
from sklearn.metrics import confusion_matrix

In [39]:
import pickle

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [48]:
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score,  recall_score, precision_score
from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

In [5]:
from my_metr import plot_roc_curve

In [6]:
from trans_rat import FillPrice, CategoriesTransformScr, RatingNormalizer, FillPriceScr


In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
rv_s = db['reviews_scrap']

In [37]:
colClust_n = ['cl_t0', 'cl_t1', 'cl_t2', 'cl_t3', 'cl_t4', 'cl_t5',  'cl_t7',
       'cl_t8', 'cl_t9', 'cl_t10', 'cl_t11', 'cl_t12', 'cl_t13', 'cl_t14',
        'cl_t16', 'cl_t17']

In [10]:
#numbers_list = pickle.load(list_unpickle)

In [35]:
clust_model_rf ={}
clust_model_gr = {}

In [40]:
for col in colClust_n:
    filename_gr = 'grboost/mgr_'+col+'.pkl'
    with open(filename_gr,'rb') as f:
        clust_model_gr[col]= pickle.load( f)
    filename_rf = 'rforest/mrf_'+col+'.pkl'
    with open(filename_rf,'rb') as f:
        clust_model_rf[col] =pickle.load( f)

In [13]:
clust_model_gr['cl_t3']

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=179, subsample=0.8, verbose=0,
              warm_start=False)

In [13]:
rest = db['restaurants']

rest_df = pd.DataFrame(list(rest.find({})))

In [14]:
rest_df.shape

(12615, 18)

### надо почистить дату

In [18]:
rest_clean = rest_df.drop(columns= ['_id','coordinates', 'display_phone', 
                                    'distance','image_url', 'name','location',
                                   'is_closed','phone', 'transactions','url'])

In [19]:
rest_clean.head(3)

,alias,categories,id,price,rating,rev_API,review_count
0,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",gDAO-4m63svPUpii5b8kww,$$,4.0,1,788
1,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",xzLAynrVb3No3ADUhMxlnQ,NaN,4.5,1,32
2,marination-ma-kai-seattle,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",IsvlaFF0EPEhh3ahTuC4xg,$$,4.0,1,897


In [26]:
#Pipeline = Pipeline([FillPriceScr, CategoriesTransformScr, RatingNormalizer ])


In [20]:
fpr = FillPriceScr()

In [21]:
rest_cl2 = fpr.transform(rest_clean)
rest_cl2.head(2)

/Users/olga/yelper/trans_rat.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X[self.priceCol][pd.isnull(X[self.priceCol])] = '$$'


,alias,categories,id,RestaurantsPriceRange2,rating,rev_API,review_count,rest_isna
0,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",gDAO-4m63svPUpii5b8kww,2,4.0,1,788,False
1,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",xzLAynrVb3No3ADUhMxlnQ,2,4.5,1,32,True


In [21]:
##Acai Bowls
mod_cat = pd.read_csv('model_cat.csv')
mod_cat.head(3)

,Unnamed: 0,0
0,Acai Bowls,0.000840
1,Active Life,0.004093
2,Afghan,0.002064


In [23]:
mod_fe = mod_cat.iloc[:,0].tolist()
mod_fe

['Acai Bowls',
 'Active Life',
 'Afghan',
 'African',
 'American (New)',
 'American (Traditional)',
 'Arabian',
 'Arcades',
 'Argentine',
 'Art Galleries',
 'Arts & Crafts',
 'Arts & Entertainment',
 'Asian Fusion',
 'Automotive',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Bars',
 'Beauty & Spas',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Beer Garden',
 'Beer Gardens',
 'Belgian',
 'Bistros',
 'Bowling',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'Brewpubs',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cafes',
 'Cafeteria',
 'Cajun/Creole',
 'Cambodian',
 'Canadian (New)',
 'Cantonese',
 'Caribbean',
 'Casinos',
 'Caterers',
 'Cheese Shops',
 'Cheesesteaks',
 'Chicken Shop',
 'Chicken Wings',
 'Chinese',
 'Chocolatiers & Shops',
 'Cocktail Bars',
 'Coffee & Tea',
 'Coffee Roasteries',
 'Colombian',
 'Comfort Food',
 'Convenience Stores',
 'Creperies',
 'Cuban',
 'Cupcakes',
 'Custom Cakes',
 'Dance Clubs',
 'Delicatessen',
 'Delis',
 'Desserts'

In [22]:
rest_cl2.info()

NameError: name 'rest_cl2' is not defined

In [126]:
with open('model_feat.pkl', 'wb') as f:
    pickle.dump(mod_fe,f)

In [127]:
with open('model_feat.pkl', 'rb') as f:
    m2 = pickle.load(f)

['Acai Bowls',
 'Active Life',
 'Afghan',
 'African',
 'American (New)',
 'American (Traditional)',
 'Arabian',
 'Arcades',
 'Argentine',
 'Art Galleries',
 'Arts & Crafts',
 'Arts & Entertainment',
 'Asian Fusion',
 'Automotive',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Bars',
 'Beauty & Spas',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Beer Garden',
 'Beer Gardens',
 'Belgian',
 'Bistros',
 'Bowling',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'Brewpubs',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cafes',
 'Cafeteria',
 'Cajun/Creole',
 'Cambodian',
 'Canadian (New)',
 'Cantonese',
 'Caribbean',
 'Casinos',
 'Caterers',
 'Cheese Shops',
 'Cheesesteaks',
 'Chicken Shop',
 'Chicken Wings',
 'Chinese',
 'Chocolatiers & Shops',
 'Cocktail Bars',
 'Coffee & Tea',
 'Coffee Roasteries',
 'Colombian',
 'Comfort Food',
 'Convenience Stores',
 'Creperies',
 'Cuban',
 'Cupcakes',
 'Custom Cakes',
 'Dance Clubs',
 'Delicatessen',
 'Delis',
 'Desserts'

In [24]:
ct = CategoriesTransformScr(mod_fe)
rn = RatingNormalizer(starCol = 'rating')
rest_cx = rn.transform(ct.transform(rest_cl2))
rest_cx.head()

,alias,id,RestaurantsPriceRange2,stars,rev_API,review_count,rest_isna,Acai Bowls,Active Life,Afghan,...,Venues & Event Spaces,Vietnamese,Waffles,Whiskey Bars,Wine & Spirits,Wine Bars,Wineries,Wraps,num_of_categ,st_over_am
0,maono-seattle,gDAO-4m63svPUpii5b8kww,2,4.0,1,788,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.016000
1,ohs-sandwiches-seattle-2,xzLAynrVb3No3ADUhMxlnQ,2,4.5,1,32,True,False,False,False,...,False,False,False,False,False,False,False,False,2,0.140621
2,marination-ma-kai-seattle,IsvlaFF0EPEhh3ahTuC4xg,2,4.0,1,897,False,False,False,False,...,False,False,False,False,False,False,False,False,3,0.016000
3,ittos-tapas-seattle-4,Dr9CxgdePozHGwU4beaYGQ,2,4.5,1,221,False,False,False,False,...,False,False,False,False,False,False,False,False,3,0.020362
4,pizzeria-credo-seattle,BeGo3TSuhF7fOv0dzp0s1g,2,4.5,1,405,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.018000


In [27]:
### i forgot to include review_count to model when train
colPred1 = rest_cx.columns[2:4].union( rest_cx.columns[6:])

In [25]:
with open('colPred213', 'rb') as f:
    colPred = pickle.load(f)

In [28]:
colPred[colPred != colPred1]

Index([], dtype='object')

In [29]:
colPred1[colPred != colPred1]

Index([], dtype='object')

In [36]:
colPred1

Index(['Acai Bowls', 'Active Life', 'Afghan', 'African', 'American (New)',
       'American (Traditional)', 'Arabian', 'Arcades', 'Argentine',
       'Art Galleries',
       ...
       'Waffles', 'Whiskey Bars', 'Wine & Spirits', 'Wine Bars', 'Wineries',
       'Wraps', 'num_of_categ', 'rest_isna', 'st_over_am', 'stars'],
      dtype='object', length=213)

In [30]:
X = rest_cx[colPred]

,Acai Bowls,Active Life,Afghan,African,American (New),American (Traditional),Arabian,Arcades,Argentine,Art Galleries,...,Wine & Spirits,Wine Bars,Wineries,Wraps,num_of_categ,price,rest_isna,review_count,st_over_am,stars


In [31]:
clt0 = clust_model_rf['cl_t0'].predict_proba(X)[:,1]

In [32]:
clt0

array([0.67499515, 0.2146956 , 0.71224975, ..., 0.43123169, 0.73124487,
       0.32225622])

In [33]:
r0 = clust_model_rf['cl_t0']

## Restaurants Prediction
make 2 dictionary, then 2 dataframe with cluster predictions

In [34]:
%%time
Y_rf = pd.DataFrame()
Y_gr = pd.DataFrame()
Y_rf['alias'] = rest_cx['alias']
Y_gr['alias'] = rest_cx['alias']
for col in colClust_n:
    Y_rf[col] = clust_model_rf[col].predict_proba(X)[:,1]
    Y_gr[col] = clust_model_gr[col].predict_proba(X)[:,1]


CPU times: user 19.7 s, sys: 1.42 s, total: 21.2 s
Wall time: 12.3 s


In [35]:
Y_gr.head(3)

,alias,cl_t0,cl_t1,cl_t2,cl_t3,cl_t4,cl_t5,cl_t7,cl_t8,cl_t9,cl_t10,cl_t11,cl_t12,cl_t13,cl_t14,cl_t16,cl_t17
0,maono-seattle,0.929968,0.992710,0.412188,0.016085,0.837059,0.919372,0.010737,0.035540,0.050482,0.009742,0.096255,2.309888e-03,0.074948,0.0,0.272277,0.037797
1,ohs-sandwiches-seattle-2,0.172442,0.302586,0.010211,0.004146,0.025425,0.049360,0.001897,0.002025,0.005802,0.000076,0.005502,1.293178e-09,0.001371,0.0,0.005165,0.000869
2,marination-ma-kai-seattle,0.953005,0.995034,0.428013,0.003022,0.780916,0.921224,0.004737,0.012592,0.024337,0.342402,0.059423,7.697930e-04,0.050621,0.0,0.290787,0.010165


In [36]:
Y_gr['rating'] = rest_cx.stars
Y_rf['rating'] = rest_cx.stars


In [38]:
Y_rf.head()

,alias,cl_t0,cl_t1,cl_t2,cl_t3,cl_t4,cl_t5,cl_t7,cl_t8,cl_t9,cl_t10,cl_t11,cl_t12,cl_t13,cl_t14,cl_t16,cl_t17,rating
0,maono-seattle,0.674995,0.899633,0.120200,0.000990,0.382802,0.447516,0.004385,0.008207,0.017935,0.000892,0.008470,0.001098,0.006509,0.000033,0.044169,0.005382,4.0
1,ohs-sandwiches-seattle-2,0.214696,0.499646,0.024044,0.000478,0.016095,0.032686,0.003058,0.000668,0.008900,0.000158,0.008909,0.005692,0.003408,0.000032,0.001514,0.000721,4.5
2,marination-ma-kai-seattle,0.712250,0.925625,0.098066,0.000551,0.396129,0.579794,0.003652,0.004784,0.006309,0.004804,0.010670,0.001601,0.006939,0.000032,0.033887,0.005368,4.0
3,ittos-tapas-seattle-4,0.576311,0.812275,0.047496,0.000632,0.211996,0.292265,0.003506,0.021161,0.010951,0.001633,0.004954,0.000692,0.026224,0.000030,0.049079,0.057404,4.5
4,pizzeria-credo-seattle,0.614117,0.815253,0.066160,0.000453,0.215199,0.225385,0.139105,0.002124,0.002613,0.000515,0.004208,0.000699,0.003090,0.000089,0.032402,0.020409,4.5


In [54]:
with open('thres_35_gr', 'rb') as f:
    th_gr35 = pickle.load(f)
with open('thres_35_rf','rb') as f:
    th_rf35 = pickle.load(f )

In [43]:
th_rf35

{'cl_t0': 0.7145996009074023,
 'cl_t1': 0.9251774972382272,
 'cl_t2': 0.09672319260149016,
 'cl_t3': 0.01,
 'cl_t4': 0.3525910562359607,
 'cl_t5': 0.3448134487670461,
 'cl_t7': 0.10512960889284231,
 'cl_t8': 0.09037281891030602,
 'cl_t9': 0.051410468261200035,
 'cl_t10': 0.16208938583548552,
 'cl_t11': 0.09687667183805175,
 'cl_t12': 0.06795537384334152,
 'cl_t13': 0.10770567432929509,
 'cl_t14': 0.01,
 'cl_t16': 0.039693625572722824,
 'cl_t17': 0.024528574099973412}

In [58]:
def predict_rating(row, colClust_n, thres):
    r = False
    for col in colClust_n:
        u_col = 'u'+col
        if row[u_col] and row[col] >= thres[col]:
            r = True
    return r

In [45]:
cl_0 = Y_rf['cl_t0'] > th_rf35['cl_t0']

In [50]:
Y_rf_bool = Y_rf.copy()
for col in colClust_n:
    Y_rf_bool[col] = Y_rf[col] > th_rf35[col]

Y_rf_bool['base3'] = Y_rf.rating < 3
Y_rf_bool['base35'] = Y_rf.rating < 3.5
Y_rf_bool['base4'] = Y_rf.rating < 4
Y_rf_bool['base4.5'] = Y_rf.rating < 4.5


NameError: name 'Y_rf' is not defined

In [51]:
Y_rf_bool[Y_rf_bool['base35']].head()

,alias,cl_t0,cl_t1,cl_t2,cl_t3,cl_t4,cl_t5,cl_t7,cl_t8,cl_t9,...,cl_t12,cl_t13,cl_t14,cl_t16,cl_t17,rating,base3,base35,base4,base4.5
101,willmotts-ghost-seattle,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,3.0,False,True,True,True
144,pho-than-brothers-seattle-6,True,True,False,False,False,True,False,False,False,...,True,False,False,False,False,3.0,False,True,True,True
152,boss-drive-in-seattle,True,True,False,False,False,False,False,True,False,...,False,False,False,False,False,3.0,False,True,True,True
171,drunkys-two-shoe-bbq-seattle-3,True,True,False,False,False,True,False,False,False,...,False,False,False,False,False,3.0,False,True,True,True
202,southside-pizza-seattle,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,3.0,False,True,True,True


In [49]:
accuracy_score(Y_rf_bool['base35'],Y_rf_bool['cl_t0'])

NameError: name 'Y_rf_bool' is not defined

### Make prediction???
1. Create user profile = set of clusters that this user care
2. Ask location 
3. Get rests from yelp for this location:
    - ask yelp api search
    - save this in database
4. Get rest predictions 
    - clean database
    - predict rests for clusters
5. Print results


In [1]:
import time
from scraping.yelp_api import *

### looking just on first 100
offset_lim = 100

In [7]:
response = search(API_KEY, DEFAULT_TERM, '416 Occidental Ave S, Seattle, WA', 50)
            

In [6]:
d0 = pd.DataFrame( response.get('businesses', None))

In [8]:
d1 = pd.DataFrame( response.get('businesses', None))

In [9]:
d0.shape, d1.shape

((50, 16), (50, 16))

In [102]:
dc = pd.concat([d0,d1], axis= 0, ignore_index=True)
dc.shape

(100, 16)

100

In [101]:
pd.concat?

In [15]:
type(response.get('businesses', None))

list

In [112]:
d = []

In [113]:
d += search(API_KEY, DEFAULT_TERM, '416 Occidental Ave S, Seattle, WA', 100).get('businesses', None)

In [114]:
len(d)

50

In [20]:
dc.columns

Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

In [52]:
from concept_proof import cleaning_data, ComboModel

In [30]:
dc.columns

Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

In [103]:
x = cleaning_data(dc, mod_fe)

/Users/olga/yelper/trans_rat.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X[self.priceCol][pd.isnull(X[self.priceCol])] = '$$'


In [41]:
clust_model_rf.keys()

dict_keys(['cl_t0', 'cl_t1', 'cl_t2', 'cl_t3', 'cl_t4', 'cl_t5', 'cl_t7', 'cl_t8', 'cl_t9', 'cl_t10', 'cl_t11', 'cl_t12', 'cl_t13', 'cl_t14', 'cl_t16', 'cl_t17'])

In [44]:
with open('colPred213', 'rb') as f:
    colPred_pr = pickle.load(f)

In [104]:
x.index

RangeIndex(start=0, stop=100, step=1)

In [69]:
x.head()

,alias,id,RestaurantsPriceRange2,stars,review_count,rest_isna,Acai Bowls,Active Life,Afghan,African,...,Venues & Event Spaces,Vietnamese,Waffles,Whiskey Bars,Wine & Spirits,Wine Bars,Wineries,Wraps,num_of_categ,st_over_am
0,the-pink-door-seattle-4,VOPdG8llLPaga9iJxXcMuQ,2,4.5,4044,False,False,False,False,False,...,False,False,False,False,False,True,False,False,3,0.018
1,taylor-shellfish-oyster-bar-seattle-2,p1TOF1XkXzLx3a-LF3lDlw,2,4.5,528,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.018
2,japonessa-sushi-cocina-seattle,L8RRAd-JZ0Bd4MER0yyX-g,2,4.0,4143,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.016
3,radiator-whiskey-seattle,ps9ZlLoLYTfK87IjW9REfg,2,4.5,1087,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.018
4,girin-korean-ssam-bar-seattle,3F5WWcqqboa-NHbYFs4LAw,2,4.0,340,False,False,False,False,False,...,False,False,False,False,False,False,False,False,3,0.016


In [45]:
pr0 = clust_model_rf['cl_t0'].predict_proba(x[colPred_pr])[:,1]

In [59]:
rf = ComboModel('rforest/mrf_', th_rf35, colPred_pr, colClust_n)

In [60]:
rf.model_thres

{'cl_t0': 0.7145996009074023,
 'cl_t1': 0.9251774972382272,
 'cl_t2': 0.09672319260149016,
 'cl_t3': 0.01,
 'cl_t4': 0.3525910562359607,
 'cl_t5': 0.3448134487670461,
 'cl_t7': 0.10512960889284231,
 'cl_t8': 0.09037281891030602,
 'cl_t9': 0.051410468261200035,
 'cl_t10': 0.16208938583548552,
 'cl_t11': 0.09687667183805175,
 'cl_t12': 0.06795537384334152,
 'cl_t13': 0.10770567432929509,
 'cl_t14': 0.01,
 'cl_t16': 0.039693625572722824,
 'cl_t17': 0.024528574099973412}

In [56]:
olka = ['cl_t0', 'cl_t2', 'cl_t9' ]

In [70]:
#rf.predict(x,olka)

In [66]:
Y = pd.DataFrame()
        #Y['alias'] = X['alias']
answer = np.array(len(x)*[False])
for col in olka:
    Y[col] = clust_model_rf[col].predict_proba(x[colPred_pr])[:,1]
    answer = answer|(Y[col]>th_rf35[col])


In [105]:
%%time
answer = rf.predict(x, olka)

CPU times: user 249 ms, sys: 49.1 ms, total: 298 ms
Wall time: 336 ms


In [107]:
len(dc[answer]['rating'])

45

In [81]:
clust_model_rf.keys()

dict_keys(['cl_t0', 'cl_t1', 'cl_t2', 'cl_t3', 'cl_t4', 'cl_t5', 'cl_t7', 'cl_t8', 'cl_t9', 'cl_t10', 'cl_t11', 'cl_t12', 'cl_t13', 'cl_t14', 'cl_t16', 'cl_t17'])

In [108]:
picky = ['cl_t0', 'cl_t1', 'cl_t2', 'cl_t3', 'cl_t4', 'cl_t5', 'cl_t7', 'cl_t8', 'cl_t9', 'cl_t10', 'cl_t11', 'cl_t12', 'cl_t13', 'cl_t14', 'cl_t16', 'cl_t17']

In [110]:
%%time
a_p = rf.predict(x, picky)

CPU times: user 1.28 s, sys: 230 ms, total: 1.51 s
Wall time: 1.79 s


In [130]:
dc[a_p].head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,the-pink-door-seattle-4,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 47.61028, 'longitude': -122.3425}",(206) 443-3241,1521.522544,VOPdG8llLPaga9iJxXcMuQ,https://s3-media1.fl.yelpcdn.com/bphoto/c_vzRF...,False,"{'address1': '1919 Post Alley', 'address2': ''...",The Pink Door,+12064433241,$$,4.5,4044,[],https://www.yelp.com/biz/the-pink-door-seattle...
1,taylor-shellfish-oyster-bar-seattle-2,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.59884, 'longitude': -122.33268}",(206) 501-4060,39.496086,p1TOF1XkXzLx3a-LF3lDlw,https://s3-media2.fl.yelpcdn.com/bphoto/he49IR...,False,"{'address1': '410 Occidental Ave S', 'address2...",Taylor Shellfish Oyster Bar,+12065014060,$$,4.5,528,[],https://www.yelp.com/biz/taylor-shellfish-oyst...
2,japonessa-sushi-cocina-seattle,"[{'alias': 'japanese', 'title': 'Japanese'}, {...","{'latitude': 47.60809, 'longitude': -122.33899}",(206) 971-7979,1168.535744,L8RRAd-JZ0Bd4MER0yyX-g,https://s3-media3.fl.yelpcdn.com/bphoto/vucCrk...,False,"{'address1': '1400 1st Ave', 'address2': '', '...",Japonessa Sushi Cocina,+12069717979,$$,4.0,4143,[restaurant_reservation],https://www.yelp.com/biz/japonessa-sushi-cocin...
3,radiator-whiskey-seattle,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 47.60888, 'longitude': -122.34036}",(206) 467-4268,1299.145186,ps9ZlLoLYTfK87IjW9REfg,https://s3-media4.fl.yelpcdn.com/bphoto/DgO6qK...,False,"{'address1': '94 Pike St', 'address2': 'Ste 30...",Radiator Whiskey,+12064674268,$$,4.5,1087,[],https://www.yelp.com/biz/radiator-whiskey-seat...
4,girin-korean-ssam-bar-seattle,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...","{'latitude': 47.59812, 'longitude': -122.33188}",(206) 257-4259,55.901362,3F5WWcqqboa-NHbYFs4LAw,https://s3-media1.fl.yelpcdn.com/bphoto/F_ZEPs...,False,"{'address1': '501 Stadium Pl S', 'address2': '...",Girin Korean Ssam Bar,+12062574259,$$,4.0,340,[pickup],https://www.yelp.com/biz/girin-korean-ssam-bar...


In [118]:
bad_p.to_html?

In [122]:
bad_p.loc[1,:].to_json()

'{"alias":"taylor-shellfish-oyster-bar-seattle-2","categories":[{"alias":"seafood","title":"Seafood"},{"alias":"raw_food","title":"Live\\/Raw Food"}],"coordinates":{"latitude":47.59884,"longitude":-122.33268},"display_phone":"(206) 501-4060","distance":39.496086001,"id":"p1TOF1XkXzLx3a-LF3lDlw","image_url":"https:\\/\\/s3-media2.fl.yelpcdn.com\\/bphoto\\/he49IR0G1wrjO1WI-fxkAg\\/o.jpg","is_closed":false,"location":{"address1":"410 Occidental Ave S","address2":"","address3":"","city":"Seattle","zip_code":"98104","country":"US","state":"WA","display_address":["410 Occidental Ave S","Seattle, WA 98104"]},"name":"Taylor Shellfish Oyster Bar","phone":"+12065014060","price":"$$","rating":4.5,"review_count":528,"transactions":[],"url":"https:\\/\\/www.yelp.com\\/biz\\/taylor-shellfish-oyster-bar-seattle-2?adjust_creative=KIqjD5y9rnzq9ev6ruaTHQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KIqjD5y9rnzq9ev6ruaTHQ"}'

In [132]:
bad_p.columns

Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

In [133]:
colShow = ['name', 'rating', 'location' , 'url']

In [135]:
bad_p[colShow].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name</th>\n      <th>rating</th>\n      <th>location</th>\n      <th>url</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>The Pink Door</td>\n      <td>4.5</td>\n      <td>{\'address1\': \'1919 Post Alley\', \'address2\': \'\'...</td>\n      <td>https://www.yelp.com/biz/the-pink-door-seattle...</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Taylor Shellfish Oyster Bar</td>\n      <td>4.5</td>\n      <td>{\'address1\': \'410 Occidental Ave S\', \'address2...</td>\n      <td>https://www.yelp.com/biz/taylor-shellfish-oyst...</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Japonessa Sushi Cocina</td>\n      <td>4.0</td>\n      <td>{\'address1\': \'1400 1st Ave\', \'address2\': \'\', \'...</td>\n      <td>https://www.yelp.com/biz/japonessa-sushi-cocin...</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Radiator Whiskey</td>\n    

In [137]:
dc[a_p][colShow]

,name,rating,location,url
0,The Pink Door,4.5,"{'address1': '1919 Post Alley', 'address2': ''...",https://www.yelp.com/biz/the-pink-door-seattle...
1,Taylor Shellfish Oyster Bar,4.5,"{'address1': '410 Occidental Ave S', 'address2...",https://www.yelp.com/biz/taylor-shellfish-oyst...
2,Japonessa Sushi Cocina,4.0,"{'address1': '1400 1st Ave', 'address2': '', '...",https://www.yelp.com/biz/japonessa-sushi-cocin...
3,Radiator Whiskey,4.5,"{'address1': '94 Pike St', 'address2': 'Ste 30...",https://www.yelp.com/biz/radiator-whiskey-seat...
4,Girin Korean Ssam Bar,4.0,"{'address1': '501 Stadium Pl S', 'address2': '...",https://www.yelp.com/biz/girin-korean-ssam-bar...
5,Il Corvo Pasta,4.5,"{'address1': '217 James St', 'address2': '', '...",https://www.yelp.com/biz/il-corvo-pasta-seattl...
6,Elliott's Oyster House,4.0,"{'address1': '1201 Alaskan Way', 'address2': '...",https://www.yelp.com/biz/elliotts-oyster-house...
7,Damn the Weather,4.0,"{'address1': '116 1st Ave S', 'address2': '', ...",https://www.yelp.com/biz/damn-the-weather-seat...
8,Intermezzo Carmine,4.5,"{'address1': '409 1st Ave S', 'address2': '', ...",https://www.yelp.com/biz/intermezzo-carmine-se...
9,Nirmal's,4.0,"{'address1': '106 Occidental Ave S', 'address2...",https://www.yelp.com/biz/nirmals-seattle?adjus...
